In [353]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# Use seaborn for pairplot
#!pip install -q seaborn

# Use some functions from tensorflow_docs
#!pip install -q git+https://github.com/tensorflow/docs

In [283]:
from tensorflow import keras
from tensorflow.keras import layers

In [284]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [285]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [497]:
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
import matplotlib.pyplot as plt

**Load Data**

In [ ]:
kc_dt_url = "https://raw.githubusercontent.com/junjiewu-mtl/glis630_final_project/master/kc_house_data.csv"

In [451]:
url = kc_dt_url
df1 = pd.read_csv(url)
df1.head()
#train_file_path = tf.keras.utils.get_file("train.csv", kc_dt_url)
# Dataset is now stored in a Pandas Dataframe

In [453]:
list(df1.columns)

['id',
 'date',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [454]:
#drop unuse features
df1 = df1.drop(["id", "date", "lat", "long", "sqft_living15", "sqft_lot15"], axis=1)

In [455]:
#covert features from numerical to factor
df1[['waterfront','view','condition','grade', 'yr_built','yr_renovated','zipcode']] = df1[['waterfront','view','condition','grade', 'yr_built','yr_renovated','zipcode']].astype('category')

## Rgression Tree Method

In [458]:
rt_train_dt = df1.sample(frac=0.8, random_state=0)
rt_test_dt = df1.drop(train_dt.index)

In [459]:
# regression tree method
rt_train_x = rt_train_dt.drop(["price"], axis=1)
rt_train_y = rt_train_dt[["price"]]

In [504]:
rt = tree.DecisionTreeRegressor(max_depth=5)
rt.fit(rt_train_x, rt_train_y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [505]:
#predicted price from the regression tree
rt_predict_price = pd.DataFrame(rt.predict(rt_test_dt.drop(["price"], axis=1)), columns=["p_price"])

In [506]:
#add row number to predicted price for merging
rt_predict_price['index'] = np.arange(len(rt_predict_price))

In [507]:
#add row number to true price for merging
rt_test_price = rt_test_dt[['price']]
rt_test_price['index'] = np.arange(len(rt_test_price))

/Users/junjiewu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [508]:
rt_price = rt_predict_price.merge(rt_test_price, left_on='index', right_on='index').round(0)


In [510]:
# mean difference between predicted price and actural price
((rt_price['p_price']-rt_price['price'])/rt_price['price']).mean()

# 0.11228089427869807

0.11203630108127514

## Regression Tree Finished HERE

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn.datasets import load_wine
from IPython.display import SVG
from graphviz import Source
from IPython.display import display

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
train_dt = df1.sample(frac=0.8, random_state=0)
test_dt = df1.drop(train_dt.index)

In [ ]:
train_stats = train_dt.describe()
train_stats.pop('price')
train_stats = train_stats.transpose()
train_stats

In [ ]:
train_labels = train_dt.pop('price')
test_labels = test_dt.pop('price')

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dt)
normed_test_data = norm(test_dt)

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=[len(train_dt.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),  
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

In [ ]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
# EPOCHS = 1000

# history = model.fit(
#   normed_train_data, train_labels,
#   epochs=EPOCHS, validation_split = 0.2, verbose=0,
#   callbacks=[tfdocs.modeling.EpochDots()])

In [ ]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

In [ ]:
hist = pd.DataFrame(early_history.history)
hist['epoch'] = early_history.epoch
hist.tail()

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
plotter.plot({'Early Stopping': early_history}, metric = "mae")
plt.ylim([0, 1000000000000000])
plt.ylabel('MAE [MPG]')

In [ ]:
plotter.plot({'Early Stopping': early_history}, metric = "mae")
plt.ylabel('MAE [price]')

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} price".format(mae))

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [price]')
plt.ylabel('Predictions [pirce]')
lims = [0, 154927]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [price]')
plt.ylabel('Predictions [pirce]')
lims = [0, 26]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [price]")
_ = plt.ylabel("Count")